### importing the dataset from kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/Kaggle/kaggle.json' '/content'

In [ ]:
import zipfile
import os

os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays

 99% 170M/172M [00:06<00:00, 25.4MB/s]
100% 172M/172M [00:06<00:00, 26.0MB/s]


In [ ]:
zip_ref = zipfile.ZipFile('bone-fracture-detection-using-xrays.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:

len(os.listdir('/tmp/archive (6)/train'))

2

### Preprocessing images


In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
generator = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

In [ ]:
Train_generator = generator.flow_from_directory(
    "/tmp/archive (6)/train",
    target_size = (224, 224),
    batch_size = 16,
    class_mode ="binary",
    subset = "training"
)

Validation_generator = generator.flow_from_directory(
    "/tmp/archive (6)/train",
    target_size = (224, 224),
    batch_size = 16,
    class_mode ="binary",
    subset = "validation"
)

Found 7091 images belonging to 2 classes.
Found 1772 images belonging to 2 classes.


In [ ]:
Train_generator.next()[0].shape

(16, 224, 224, 3)

In [ ]:
Validation_generator.next()[0].shape

(16, 224, 224, 3)

### Developing the model

In [ ]:
from tensorflow import keras

In [ ]:
base_model = keras.applications.Xception(
    weights=None,
    input_shape=(224, 224, 3),
    include_top=False)

In [ ]:
base_model.trainable = True

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=True)
x = keras.layers.GaussianNoise(0.25)(x)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(512,activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.GaussianNoise(0.25)(x)
x = keras.layers.Dropout(0.25)(x)
outputs = keras.layers.Dense(1, activation = "sigmoid")(x)
model = keras.Model(inputs, outputs)

In [ ]:
from keras import callbacks

In [ ]:
earlyStopping = keras.callbacks.EarlyStopping(
    patience=7,
)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
model.fit(Train_generator, epochs=5,
          callbacks=[earlyStopping,model_checkpoint_callback ],
          validation_data=Validation_generator)

Epoch 1/5
444/444 [==============================] - ETA: 0s - loss: 0.6517 - accuracy: 0.6579

444/444 [==============================] - 199s 350ms/step - loss: 0.6517 - accuracy: 0.6579 - val_loss: 0.8015 - val_accuracy: 0.5293
Epoch 2/5
444/444 [==============================] - ETA: 0s - loss: 0.3854 - accuracy: 0.8280

444/444 [==============================] - 152s 343ms/step - loss: 0.3854 - accuracy: 0.8280 - val_loss: 1.3540 - val_accuracy: 0.5688
Epoch 3/5
444/444 [==============================] - ETA: 0s - loss: 0.2087 - accuracy: 0.9158

444/444 [==============================] - 152s 343ms/step - loss: 0.2087 - accuracy: 0.9158 - val_loss: 1.2125 - val_accuracy: 0.5942
Epoch 4/5
444/444 [==============================] - ETA: 0s - loss: 0.1130 - accuracy: 0.9612

444/444 [==============================] - 153s 344ms/step - loss: 0.1130 - accuracy: 0.9612 - val_loss: 1.3113 - val_accuracy: 0.6044
Epoch 5/5
444/444 [==============================] - 128s 289ms/step - loss: 0.1019 - accuracy: 0.9650 - val_loss: 1.6938 - val_accuracy: 0.5412


In [ ]:
model.evaluate(Train_generator)

444/444 [==============================] - 37s 82ms/step - loss: 0.0539 - accuracy: 0.9766


[0.0539282001554966, 0.9765900373458862]

In [ ]:
Test_Generator = generator.flow_from_directory(
    "/tmp/archive (6)/val",
    target_size = (224, 224),
    batch_size = 16,
    class_mode ="binary",
    subset = "training"
)

Found 480 images belonging to 2 classes.


In [ ]:
model.evaluate(Test_Generator)

30/30 [==============================] - 3s 82ms/step - loss: 2.3392 - accuracy: 0.6042


[2.3391671180725098, 0.6041666865348816]